This notebook is to create (manageable sized) cutouts of images from a set of parent images.

In [1]:
from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.utils.data import download_file
from astropy.wcs import WCS

Define the parent images

In [2]:

# the path to the directory holding the parent images
# WARNING: this is currently specific to Steve's machine 
parent_image_dir = '/Users/sw376/Dropbox/Research/data/images/jwst/ceers/images/0.2'

# list of filters to use
filters = ['f115w', 'f150w', 'f200w', 'f277w', 'f356w', 'f444w']
# filters = ['f115w']

# dictionary of images with filter code as the name
parent_images = {filter: f'ceers_nircam1_{filter}_v0.2_i2d.fits' for filter in filters}

Explore the first image

In [3]:

# open the first image and get some information, including the dimensions

filename = f'{parent_image_dir}/{parent_images[filters[0]]}'

# the structure of i2d files are described here:
# https://jwst-pipeline.readthedocs.io/en/latest/jwst/data_products/science_products.html#resampled-2-d-data-i2d-and-s2d

# open the i2d image file
hdul = fits.open(filename)

# get info
hdul.info()

# grab the extension holding the science image
hdu = hdul['SCI']

# print the shape of the science image
print(hdu.data.shape)

# print the header keywords
for key, value in hdu.header.items():
    print(key, value)

# grab the wcs
wcs = WCS(hdu.header)

Filename: /Users/sw376/Dropbox/Research/data/images/jwst/ceers/images/0.2/ceers_nircam1_f115w_v0.2_i2d.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     369   ()      
  1  SCI           1 ImageHDU        75   (10600, 4800)   float32   
  2  ERR           1 ImageHDU        10   (10600, 4800)   float32   
  3  CON           1 ImageHDU         9   (10600, 4800)   int32   
  4  WHT           1 ImageHDU         9   (10600, 4800)   float32   
  5  VAR_POISSON    1 ImageHDU         9   (10600, 4800)   float32   
  6  VAR_RNOISE    1 ImageHDU         9   (10600, 4800)   float32   
  7  VAR_FLAT      1 ImageHDU         9   (10600, 4800)   float32   
  8  HDRTAB        1 BinTableHDU    816   96R x 403C   [23A, 5A, 3A, 48A, 7A, 13A, 5A, 5A, 7A, 10A, 4A, L, D, D, D, D, 4A, 18A, 57A, 22A, 3A, D, 20A, D, 10A, 12A, 23A, 23A, 26A, 11A, 5A, 3A, 3A, 2A, 1A, 2A, 1A, L, 12A, 23A, 2A, 26A, 20A, 27A, 10A, K, L, L, L, L, 5A, 1A, 5A, D, D, D, D, D, D, D, D

Set DATE-AVG to '2022-06-21T19:19:05.420' from MJD-AVG.
Set DATE-END to '2022-06-21T20:27:34.055' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to   -36.739053 from OBSGEO-[XYZ].
Set OBSGEO-H to 1725216481.136 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


#### Define the position and size of the cutout

In [4]:
position = (500, 300)
size = (200, 200)
output_dir = '../test_data/'

#### Create cutouts

In [5]:

# loop over parent images
for filter, parent_image in parent_images.items(): 

    print(filter)

    # create new hdul with empty primary HDU
    cutout_primary_hdu = fits.PrimaryHDU()
    cutout_hdul = fits.HDUList([cutout_primary_hdu])

    # open the i2d image file
    hdul = fits.open(f'{parent_image_dir}/{parent_image}')

    for ext in ['SCI', 'ERR', 'WHT']:    

        # grab the extension holding the science image
        hdu = hdul[ext]

        # Make the cutout, including the WCS
        cutout = Cutout2D(hdu.data, position=position, size=size, wcs=wcs)

        # Create the hdu for this extention
        cutout_hdu = fits.ImageHDU(data=cutout.data, name=ext)
        
        # Update the FITS header with the cutout WCS
        cutout_hdu.header.update(cutout.wcs.to_header())

        # Append the hdu to the hdu list
        cutout_hdul.append(cutout_hdu)

    # Write the cutout to a new FITS file
    cutout_filename = f"{output_dir}/{filter}_i2d.fits"
    cutout_hdul.writeto(cutout_filename, overwrite=True)



f115w


FileNotFoundError: [Errno 2] No such file or directory: '../../test_data//f115w_i2d.fits'